In [1]:
# pip install transformers


In [2]:
 #pip install datasets

In [3]:
#pip install sentencepiece

In [4]:
#pip install rouge_score

In [5]:
#pip install wandb (Use Weights and Biases API to build better models faster(Optional))

In [6]:
WANDB_INTEGRATION = True
if WANDB_INTEGRATION:
    import wandb

    wandb.login()

wandb: Currently logged in as: seemapatil1585. Use `wandb login --relogin` to force relogin


In [7]:
import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

from tabulate import tabulate
import nltk
from datetime import datetime

In [8]:
#model_name to indicate the specific pretrained model used
pretrained_model_name = "sshleifer/distilbart-xsum-12-3"


# Load the pretrained model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

# Set model parameters or use the default
# print(model.config)

# Tokenization parameters
encoder_max_len = 256
decoder_max_len = 64


In [9]:
# Load dataset from the CSV file
import pandas as pd
geographical_data_df = pd.read_csv('Geographical data_2.csv', encoding='latin-1')

In [10]:
# Reduce the dataset to 200 rows
geographical_data_df = geographical_data_df.head(200)

In [11]:
import pandas as pd
from datasets import Dataset

# Convert DataFrame to Dataset
geographical_data = Dataset.from_pandas(geographical_data_df)


def flatten_example(example):
    return {
        "CityDescription": example["City Description"],
        "FeatureExtraction": example["Feature Extraction"],
    }

def filter_samples(example):
    CityDescription = []
    FeatureExtraction = []
    for desc, feat in zip(example["City Description"], example["Feature Extraction"]):
        if len(desc) > 0:
            CityDescription.append(desc)
            FeatureExtraction.append(feat)
    return {"CityDescription": CityDescription, "FeatureExtraction": FeatureExtraction}

# Apply transformations to the dataset
geographical_data = geographical_data.map(flatten_example)
geographical_data = geographical_data.map(filter_samples, batched=True)

# Split the dataset into train and validation sets
train_data_txt, validation_data_txt = geographical_data.train_test_split(test_size=0.1).values()

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [12]:
def preprocess_batch(batch, tokenizer, max_source_length, max_target_length):
    src, tgt = batch["CityDescription"], batch["FeatureExtraction"]
    src_tokenized = tokenizer(
        src, padding="max_length", truncation=True, max_length=max_source_length
    )
    tgt_tokenized = tokenizer(
        tgt, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in src_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in tgt_tokenized["input_ids"]
    ]
    return batch


In [13]:
train_data = train_data_txt.map(
    lambda batch: preprocess_batch(
        batch, tokenizer, encoder_max_len, decoder_max_len
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

In [14]:
validation_data = validation_data_txt.map(
    lambda batch: preprocess_batch(
        batch, tokenizer, encoder_max_len, decoder_max_len
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [15]:
import nltk
import numpy as np
from datasets import load_metric
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer

nltk.download("punkt", quiet=True)

metric = load_metric("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels



<ipython-input-15-c2bc784d880c>:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [17]:
def calculate_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [18]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=1,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,  # demo
    per_device_eval_batch_size=4,
    # learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    save_total_limit=3,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=calculate_metrics,
)

Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (urllib3 1.26.16 (c:\users\seema.patil03\anaconda3\lib\site-packages), Requirement.parse('urllib3<=1.26.7,>=1.23')).


In [19]:
trainer.evaluate()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 5.641493797302246,
 'eval_rouge1': 4.4876,
 'eval_rouge2': 0.0,
 'eval_rougeL': 4.502,
 'eval_rougeLsum': 4.4704,
 'eval_gen_len': 17.65,
 'eval_runtime': 78.8865,
 'eval_samples_per_second': 0.254,
 'eval_steps_per_second': 0.063}

In [20]:
trainer.train()

C:\Users\seema.patil03\Anaconda3\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=45, training_loss=4.650253634982639, metrics={'train_runtime': 813.2117, 'train_samples_per_second': 0.221, 'train_steps_per_second': 0.055, 'total_flos': 55724212224000.0, 'train_loss': 4.650253634982639, 'epoch': 1.0})

In [21]:
trainer.evaluate()

{'eval_loss': 3.1402130126953125,
 'eval_rouge1': 36.9058,
 'eval_rouge2': 22.3935,
 'eval_rougeL': 34.8485,
 'eval_rougeLsum': 34.5047,
 'eval_gen_len': 18.95,
 'eval_runtime': 79.0655,
 'eval_samples_per_second': 0.253,
 'eval_steps_per_second': 0.063,
 'epoch': 1.0}

In [22]:
def generate_summary(test_data, model):
    inputs = tokenizer(
        test_data["CityDescription"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_len,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

model_before_fine_tuning = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name)

test_samples = validation_data_txt.select(range(10))

features_before_fine_tuning = generate_summary(test_samples, model_before_fine_tuning)[1]
features_after_fine_tuning = generate_summary(test_samples, model)[1]

C:\Users\seema.patil03\Anaconda3\lib\site-packages\transformers\generation\utils.py:1288: UserWarning: Using `max_length`'s default (62) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [23]:
print("\nTarget Feature:\n")
print(
    tabulate(list(enumerate(test_samples["FeatureExtraction"])), headers=["Id", "Target Feature"])
)
print("\nCity Description:\n")
print(tabulate(list(enumerate(test_samples["CityDescription"])), headers=["Id", "City Description"]))


Target Feature:

  Id  Target Feature
----  ------------------------------------------------------------------------------------------------------
   0  Serene and scenic riverfront, educational institutions and Baidyabati Railway Station.
   1  Agricultural activities, Handloom industry
   2  Coal mines, ancient temples and monuments.
   3  Historical and religious sites, agricultural significance.
   4  pristine beaches, historical sites, agricultural production
   5  Tea gardens, cultural heritage
   6  Historical significance, agricultural activities.
   7  Historical and religious significance, agricultural activities.
   8  Historical significance and architectural marvels, rich cultural heritage and hub for classical music.
   9  Historical significance, architectural marvels

City Description:

  Id  City Description
----  ------------------------------------------------------------------------------------------------------------------------------------------------------------